- 닉네임 추출
  - 텍스트에 ','가 없으면 당일 대화 날짜를 나타냄
  - 텍스트에 ',' 가 있으면 대화 내용이다.
      - 추출 내용을 봐서는 첫번째 ',' 뒤에 대화 상대가 나온다
      - 첫번째 ',' 앞에는 날짜가 나온다.
      - 첫번째 ':' 뒤에는 대화 내용이 나온다.

#### 문자 예외

## 모델 install

In [162]:
# !pip3 install krwordrank
!pip3 install gensim

In [165]:
!pip3 install gensim==3.6.0 # from gensim.summarization import summarize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 8.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gensim: filename=gensim-3.6.0-cp311-cp311-macosx_11_0_x86_64.whl size=23218278 sha256=e80b006773ee3bb4b9bed9c4596c9b54a38463dedaafee46b02a3d89c4143b98
  Stored in directory: /Users/goodyoung/Library/Caches/pip/wheels/6b/84/4d/ea7977f42f89ebaa24f5702cc2d5013300934c7641d48c1521
Successfully built gensim


In [229]:
# !pip3 install keybert
# !pip3 install kiwipiepy

### TXT 인풋

In [48]:
import re

In [49]:
from datetime import datetime

In [50]:
class Conversation:
    def __init__(self):
        self.conversation = [] # 총 대화
        self.date = [] # 대화 날짜
        self.mergeConversation = [] # 하루 대화 총합
        self.dailyMessageList = [] # 하루대화
        self.dailyMessageStr = ''
        self.monthFlag = ''
        
    def isMessage(self,line:str):
        """ 
        메세지 특성 판별 
        1. 대화 시작
        2. 그 전과 이어진 대화인지
        3. 시작한 날짜인지
        """
        pattern1 = r'\d{4}\. \d{1,2}\. \d{1,2}\. [오후|오전]* \d{1,2}:\d{1,2}'
        pattern2 = r'^\d{4}\년 \d{1,2}\월 \d{1,2}\일 [월화수목금토일]요일+$' # ^는 문자열 시작 $는 문자열 끝부분
        if re.match(pattern1, line): # 대화 시작이다.
            return 1
        elif re.match(pattern2, line): # 날짜이다.
            return 2
        else: # 추가적 대화이다.
            return 0
            
    def messagePreprocess(self,message:str):
        """ 
        대화 전처리 
        1. 웃음, 울음 (ㅋㅋ, ㅎㅎ)
            - ㅋㅋㅋㅋ, ㅎㅎㅎㅎㅎ
        2. 느낌표 (!) , 물음표 (?)
            - !, !!!!!!!, !?!?
        3. 쉼표(,), 점(.)
            - ……. ,,,,,,
        4. 이모티콘 타입 1 (입 중심)
            - :) :D
        5. 이모티콘 타입 1 (눈 중심)
            - ^^, ㅜㅜ
        """
        pattern1 = "(\(.+?\))|([ㄴㅋㅎㅇzㅠㅜ!?~]{3,})|[,.]{2,}|([;:]{1}[\^'-]?[)(DPpboOX] )|([>ㅜㅠㅡ@+\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@+<];*)"
        message = re.sub(pattern1,'',message)
        if line: #전처리 결과 대화가 있을 경우
            return message.strip()
        else:
            return '' #or None
    
    def dailyConversation(self):
        """ 하루 대화 """
        if len(self.date) != 0:
            totalConversation = [self.date[-1], self.dailyMessageList,self.dailyMessageStr]
            self.mergeConversation.append(totalConversation) # 대화 넣기
            self.dailyMessageList = [] # 그날 대화 초기화
            self.dailyMessageStr = ''
        
    def conversationSplit(self,line:str):
        """ 대화 분리 """
        num = self.isMessage(line)
        if num == 1: # 대화 시작
            line_set = line.split(',', maxsplit=1)
            line_set[0] = self.changeTimeType(line_set[0]) # 날짜 형식 변환 "%Y-%m-%d %H:%M"로
            pop_line = line_set.pop()
            name, message = pop_line.split(':', maxsplit=1) # 대화 이름, 대화 내용 분리
            message = self.messagePreprocess(message) # 대화 전처리
            if message:
                self.dailyMessageList.append(message) #그 날 대화 종합 
                self.dailyMessageStr = self.dailyMessageStr + ' ' + message

            line_sets = line_set + [name.strip(), message]
            self.conversation.append(line_sets)
        elif num == 2: # 날짜
            self.dailyConversation() #그 전날 대화 총합
            self.date.append(self.changeDateType(line))
            
        else: # 추가적 대화
            self.conversation[-1].append(line)

    def changeDateType(self,line:str):
        """ 대화 날짜 타입 변경 """
        line = '-'.join(re.findall(r'\d+', line))
        dt = datetime.strptime(line, "%Y-%m-%d").strftime("%Y-%m-%d")
        return dt

    def changeTimeType(self,line:str):
        """ 대화 시간 타입 변경 """
        line = line.replace("오후","PM").replace("오전","AM")
        dt = datetime.strptime(line, "%Y. %m. %d. %p %I:%M").strftime("%Y-%m-%d %H:%M")
        return dt

In [51]:
conv = Conversation()
f = open('test2.txt','r')
data = f.readlines()
for line in data[2:]:
    if line !='\n':
        conv.conversationSplit(line.replace('\n','').strip())

In [52]:
conv.date[-3:]

['2023-09-23', '2023-10-10', '2023-10-11']

In [58]:
conv.mergeConversation[-1][2]

' 나는 인터스텔라 봤어 난 안봤는데 재밌어? 스포해줄까? ㄱㄱ 인터스텔라는 도서관에서 아빠가 나와 아니 엄마가 아니 뭐였지 동생 아니 여 동생 이해했어? ㅇㅎ 나만 대화하니? 이게 다 중요하다고 딴걸로 가자 그러다가 그럴까? 어렵다 인터스텔라 센과 치히로의 행방불명 봤ㅇ? 센과 치히로 봤지 센 치히로 나 지브리 좋아하거든 지브리 좋아해? 나는 센과 치히로에 나오는 돼지를 보고 울었어 너무 슬프게 생겨서 그거 돼지 비하아니야? 신고 신고하지마 나는 돼지를 좋아하며 존경하며 동경해 난 거기 하쿠가 돼지고기 먹고싶네 좀 멋있더라 응 오 하쿠는 남자야 여자야? 존잘 남자 남자구나 몰랐어 나도 저렇게 태어날걸 하쿠는 여자 닮았더라고 아쉬움이 남아 아하? 그럴 순 없어 우리 조금만 더 대화해야해 어제 뭐였ㅈ 체육대회 갔어? 어제 전 종목 다 나갔지 온 몸이 다 쑤셔 오 심지어 이겼어? 후면삼각근까지 알 배겼어 오 이겼지 그래서 진짜? 배민상품권 준대 체육대회 이겼구나?! 개이득 나도 갈게 그땐 언제 올건데 당신 난 시간이 되면 체육대회에서 술도 마시고 닭발도 먹고 삼겹살도 먹었어 돈도 내고? 지리지 만원 냈는데 가성비 지리네 난 한입 밖에 안 먹었어 어머 누가 다 먹었을까 술은 누가 마셨어 술도 다 마셨나 난 술 한 잔 마셨어 소오주? 하이볼 맛있었겠다 맛있더라 이만하면 될 것 같은데 핫식스랑 섞으면 좀 맛있어짐 카페인 덩어리네 ㅇㅈ 곧 뒤짐 이걸 한 번 돌려볼게 ㅎㅎ'

In [17]:
s = ['예쁜 영상과 아름다운 음악 꿈을 쫒는 두사람의 선택이 달랐다면 어땠을까 상상하는 장면이 인상깊었다 쓸쓸하지만 현실적인 사랑이랄까',
 '음악도 좋고 미아와 세바스티안의 아름다운 사랑과 예술에 대한 열정이 감동적이었습니다 재즈음악을 사랑하고 뮤지컬을 좋아하는 사람들에게 강추합니다',
 '생각보다 굉장히 재미있는 뻔한 결말도 아니고 아름다운 음악과 현실적인 스토리구성 모두에게 와닿을법한 울림들이 차 좋았어요 추천',
 '먼저 음악이 너무 좋고아름다운 영상미만으로도 최고네요 아름답지만 짠내도 나구요 별 생각없이 봤는데 강추입니다 영화보고 계속 음악이 귀에 맴돌아요',
 '사랑 꿈 현실 모든걸 다시한번 생각하게 하는 영화였어요 영상미도 너무 예쁘고 주인공도 예쁘고 내용도 아름답네요ㅠㅠ 인생 영화',
 '너무 좋은 영화 스토리는 비숫한것같아요 그래도 음악 영상 이루어지지않는 사랑을 더 매력적으로 전달한영화인것같아요 보고나서도 여운이 남는',
 '노래도 좋고 영상미도 좋고 그리고 배우들 연기까지 정말 좋았어요 개인적으로 뮤지컬 형식 영화를 안좋아하는 편인데 재밌게 봤습니다',
 '16년 최고의영화 인생영화입니다 영상미 색감 음악 감정선 다좋았는데 엔딩이 참현실적이네요 ㅎㅎ 참 공감되고 감동받았습니다 씁쓸하니 정말잘봤어요',
 '사실 두번째 보는 영화입니다 영상 편집과 음악이 너무 좋아요 어떻게 보면 너무나 현실적일 수 있는 결말이 슬프기하지만 아름답습니다',
 '영화사에 남을 최고의 뮤지컬영화입니다 음악과 영상이 너무 아름답고 두 주연배우의 연기는 매우 감동적입니다 무조건 보세요 최고']

### 요약 1..

In [13]:
v = sum.split()

In [24]:
from krwordrank.word import summarize_with_keywords
sum = """근데 난 어디든 상관이 없어서 노바 와 나 외로워 아는 애가없어.
진짜로 고민이네 너무 그 큐빅 회장형 보기가 그런데 어차피 큐빅 사람 너무 많아서 누군지 다 모를듯 노바 왜 감 나입니다."""
keywords = summarize_with_keywords()
keywords

{}

In [10]:
from krwordrank.word import summarize_with_keywords
keywords = summarize_with_keywords(conv.mergeConversation[24][1])
keywords
#conv.mergeConversation[1][1]

{'자구': 1.072785020620934, 'ㅎㅎ': 1.0, '컴구': 0.6815824890118777}

In [156]:
conv.mergeConversation[1][1]

['건영',
 '어디갈거야',
 '동아리??',
 '예아',
 '진짜 모르겠네',
 '노바가',
 '어디였지',
 '작년 1등',
 '한 동아리?',
 '아하?!',
 '으',
 '저거',
 '오늘까지야?',
 'ㄴㄴ',
 '근데',
 '난 어디든 상관이 없어서',
 '노바 와 나 외로워',
 '아는 애가없어',
 '진짜로',
 '고민이네',
 '너무',
 '그 큐빅 회장형 보기가 그런데',
 '어차피 큐빅 사람',
 '너무 많아서',
 '누군지 다 모를듯',
 '노바 왜 감',
 '나?',
 '엉',
 '끌렸어?',
 '그 소개하러 올때',
 '1등했다길래',
 '어',
 '가야겠다',
 '이름도 멋있고',
 '하',
 '고민해바',
 '오키',
 '으으ㅡ',
 '아니',
 '그럼',
 '약속 다시 잡아야함?',
 '그러지않을까?',
 '짜증나',
 '낼 안되면 화욜에 되도록',
 '추진해야겠다',
 '화욜',
 '?',
 '아',
 '이번주?',
 '아 그리',
 '치',
 '낼모레',
 '너가 선빵쳤구나',
 '빨리',
 '해버려야지',
 '안구ㅣ차낱',
 '맞긴해',
 '언제 말하나싶었어 쟤네',
 '아 그러네?',
 '공업 화학과는',
 '처음 듣는데',
 '나도',
 '웬만한 건 다아는데',
 '진짜 생소하네',
 '화학과랑 다른가',
 '화공이랑 비슷한가',
 '형진',
 '인디노래 추천좀',
 '좀 빡센거',
 'ㅎㅎ',
 '빡센 인디? Z',
 '어떻게 빡센거',
 '아',
 '박자감',
 'ㅎㅎ',
 '리듬',
 '오케이',
 '기대할게',
 '안돼',
 '기대금지',
 'ㅇㅋ…',
 '검정치마를 아시나요',
 '몰라',
 '오잉',
 '?',
 '생각보다 안 유명하구나',
 '뭐야',
 '2288만회?',
 '아니 내가 잘 몰라',
 '모른다면',
 '샵검색: #everything 검정치마',
 '일단 이걸',
 '드럼',
 '시작이네',
 '박자감은 잘모르겠는데',
 '그냥 몽환적인 노래야',
 '좋지',
 '혁오',
 '느낌',
 '그치

### 요약 2..

In [166]:
from gensim.summarization import summarize

In [179]:
conv.mergeConversation[1][2]

'건영어디갈거야동아리??예아진짜 모르겠네노바가어디였지작년 1등한 동아리?아하?!으저거오늘까지야?ㄴㄴ근데난 어디든 상관이 없어서노바 와 나 외로워아는 애가없어진짜로고민이네너무그 큐빅 회장형 보기가 그런데어차피 큐빅 사람너무 많아서누군지 다 모를듯노바 왜 감나?엉끌렸어?그 소개하러 올때1등했다길래어가야겠다이름도 멋있고하고민해바오키으으ㅡ아니그럼약속 다시 잡아야함?그러지않을까?짜증나낼 안되면 화욜에 되도록추진해야겠다화욜?아이번주?아 그리치낼모레너가 선빵쳤구나빨리해버려야지안구ㅣ차낱맞긴해언제 말하나싶었어 쟤네아 그러네?공업 화학과는처음 듣는데나도웬만한 건 다아는데진짜 생소하네화학과랑 다른가화공이랑 비슷한가형진인디노래 추천좀좀 빡센거ㅎㅎ빡센 인디? Z어떻게 빡센거아박자감ㅎㅎ리듬오케이기대할게안돼기대금지ㅇㅋ…검정치마를 아시나요몰라오잉?생각보다 안 유명하구나뭐야2288만회?아니 내가 잘 몰라모른다면샵검색: #everything 검정치마일단 이걸드럼시작이네박자감은 잘모르겠는데그냥 몽환적인 노래야좋지혁오느낌그치그치오이거 어디서 많이들어본 것 같은데유명해서 그런가이게 젤 히트친거라아마 들어봣을거야좋다허회경알아?그럼유노래 2개밖에 모르긴한데허회경 - 그렇게 살아가는 것이거야?어들켰다아 한개더아무것도 상관없어검정치마에서 인디 플리로 넘어가버렸다ㅎㅎㅋㅋ여성분이시네맞어 나도 남자인줄 ㅋㅋ새벽에 들으면 이제 못 벗어나무서운 장르구만이제 알고리즘에 지배당할 일만 남았다구내일 그대로 하겠는데?ㅎㅋ개뀰아니개 에반데족보랑 똑같은게 말이 됨?아 억울해똑같다고?거의 똑같아왜 난 다르다고느꼈지그래??나이수1번 똑같고2번 숫자만 바꿨고오족보 4번이랑 3번이랑 숫자만 다르고2주차 말하는거지으1주차이거이거 노바 가야겠는데누나한테 고마워섴ㅋ노바 사람도별로없어서족보는 다 잘 챙겨줄듯사람 없어??딴데보다는큐빅이나 네넷보단 확실히 적으니까아 그려?큐빅 사람 적은 줄 알았네말하는거보면 꽤많아보이던데'

In [197]:
s = '''건영어디갈거야동아리??예아진짜 모르겠네노바가어디였지작년 1등한 동아리?아하?!으저거오늘까지야?ㄴㄴ근데난 어디든 상관이 없어서노바 와 나 외로워아는 애가없어진짜로고민이네너무그 큐빅 회장형 보기가 
그런데어차피 큐빅 사람너무 많아서누군지 다 모를듯노바 왜 감나?엉끌렸어?그 소개하러 올때1등했다길래어가야겠다이름도 멋있고하고민해바오키으으ㅡ아니그럼약속 다시 잡아야함?그러지않을까?짜증나낼 안되면 화욜에 되도록추진해야겠다화욜?아이번주?아 그리치낼모레너가 선빵쳤구나빨리해버려야지안구ㅣ차낱맞긴해언제 말하나싶었어 쟤네아 그러네?공업 화학과는처음 듣는데나도웬만한 건 다아는데진짜 생소하네화학과랑 다른가화공이랑 비슷한가형진인디노래 추천좀좀 빡센거ㅎㅎ빡센 인디? Z어떻게 빡센거아박자감ㅎㅎ리듬오케이기대할게안돼기대금지ㅇㅋ…검정치마를 아시나요몰라오잉?생각보다 안 유명하구나뭐야2288만회?아니 내가 잘 몰라모른다면샵검색: 
#everything 검정치마일단 이걸드럼시작이네박자감은 잘모르겠는데그냥 몽환적인 노래야좋지혁오느낌그치그치오이거 어디서 많이들어본 것 같은데유명해서 그런가이게 젤 히트친거라아마 들어봣을거야좋다허회경알아?그럼유노래 2개밖에 모르긴한데허회경 - 그렇게 살아가는 것이거야?
어들켰다아 한개더아무것도 상관없어검정치마에서 인디 플리로 넘어가버렸다ㅎㅎㅋㅋ여성분이시네맞어 나도 남자인줄 ㅋㅋ새벽에 들으면 이제 못 벗어나무서운 장르구만이제 알고리즘에 지배당할 일만 남았다구내일 그대로 하겠는데?ㅎㅋ개뀰아니개 에반데족보랑 똑같은게 말이 됨?아 억울해똑같다고?
거의 똑같아왜 난 다르다고느꼈지그래??나이수1번 똑같고2번 숫자만 바꿨고오족보 4번이랑 3번이랑 숫자만 다르고2주차 말하는거지으1주차이거이거 노바 가야겠는데누나한테 고마워섴ㅋ노바 사람도별로없어서족보는 다 잘 챙겨줄듯사람 없어??딴데보다는큐빅이나 네넷보단 확실히 적으니까아 그려?큐빅 사람 적은 줄 알았네말하는거보면 꽤많아보이던데'''

In [190]:
article = '''
이스라엘이 오는 23일부터 실내 마스크 의무도 해제하기로 발표한 가운데 방대본은 "실외 마스크 착용 의무를 해제하더라도 실내 마스크는 상당 기간 착용 의무를 유지할 방침"이라고 강조했다.
김 팀장은 계절적으로 여름이 다가오는 가운데 문 닫고 에어컨을 이용하는 습관, 3밀 환경과 지하철·버스를 많이 이용하는 생활을 고려할 때 실내마스크 해제 "시기상조"라며 "굉장히 신중한 검토가 필요한 상황"이라고 말했다.
그는 "향후 실외 마스크를 해제하더라도 실내에서는 고위험군을 보호하고 유증상자 등 감염될 수 있는 분을 차단할 수 있도록 상당 기간 착용 권고할 예정"이라고 밝혔다.
'''

In [223]:
summarize(s,word_count = 30)

'건영어디갈거야동아리??예아진짜 모르겠네노바가어디였지작년 1등한 동아리?아하?!으저거오늘까지야?ㄴㄴ근데난 어디든 상관이 없어서노바 와 나 외로워아는 애가없어진짜로고민이네너무그 큐빅 회장형 보기가 \nZ어떻게 빡센거아박자감ㅎㅎ리듬오케이기대할게안돼기대금지ㅇㅋ…검정치마를 아시나요몰라오잉?생각보다 안 유명하구나뭐야2288만회?아니 내가 잘 몰라모른다면샵검색: '

### 요약 3..

In [25]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

/Users/goodyoung/Desktop/대학교/korean/korean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [240]:
conv.mergeConversation[1][2]

' 건영 어디갈거야 동아리?? 예아 진짜 모르겠네 노바가 어디였지 작년 1등 한 동아리? 아하?! 으 저거 오늘까지야? ㄴㄴ 근데 난 어디든 상관이 없어서 노바 와 나 외로워 아는 애가없어 진짜로 고민이네 너무 그 큐빅 회장형 보기가 그런데 어차피 큐빅 사람 너무 많아서 누군지 다 모를듯 노바 왜 감 나? 엉 끌렸어? 그 소개하러 올때 1등했다길래 어 가야겠다 이름도 멋있고 하 고민해바 오키 으으ㅡ 아니 그럼 약속 다시 잡아야함? 그러지않을까? 짜증나 낼 안되면 화욜에 되도록 추진해야겠다 화욜 ? 아 이번주? 아 그리 치 낼모레 너가 선빵쳤구나 빨리 해버려야지 안구ㅣ차낱 맞긴해 언제 말하나싶었어 쟤네 아 그러네? 공업 화학과는 처음 듣는데 나도 웬만한 건 다아는데 진짜 생소하네 화학과랑 다른가 화공이랑 비슷한가 형진 인디노래 추천좀 좀 빡센거 ㅎㅎ 빡센 인디? Z 어떻게 빡센거 아 박자감 ㅎㅎ 리듬 오케이 기대할게 안돼 기대금지 ㅇㅋ… 검정치마를 아시나요 몰라 오잉 ? 생각보다 안 유명하구나 뭐야 2288만회? 아니 내가 잘 몰라 모른다면 샵검색: #everything 검정치마 일단 이걸 드럼 시작이네 박자감은 잘모르겠는데 그냥 몽환적인 노래야 좋지 혁오 느낌 그치그치 오 이거 어디서 많이 들어본 것 같은데 유명해서 그런가 이게 젤 히트친거라 아마 들어봣을거야 좋다 허회경 알아? 그럼유 노래 2개밖에 모르긴한데 허회경 - 그렇게 살아가는 것 이거야 ? 어 들켰다 아 한개더 아무것도 상관없어 검정치마에서 인디 플리로 넘어가버렸다 ㅎㅎ ㅋㅋ 여성분이시네 맞어 나도 남자인줄 ㅋㅋ 새벽에 들으면 이제 못 벗어나 무서운 장르구만 이제 알고리즘에 지배당할 일만 남았다구 내일 그대로 하겠는데? ㅎㅋ 개뀰 아니 개 에반데 족보랑 똑같은게 말이 됨? 아 억울해 똑같다고? 거의 똑같아 왜 난 다르다고느꼈지 그래?? 나이수 1번 똑같고 2번 숫자만 바꿨고 오 족보 4번이랑 3번이랑 숫자만 다르고 2주차 말하는거지 으 1주차 이거이거 노바 가야겠는데 누나한테 고마워섴ㅋ 노바 사람도

In [241]:
text = ' 건영 어디갈거야 동아리?? 예아 진짜 모르겠네 노바가 어디였지 작년 1등 한 동아리? 아하?! 으 저거 오늘까지야? ㄴㄴ 근데 난 어디든 상관이 없어서 노바 와 나 외로워 아는 애가없어 진짜로 고민이네 너무 그 큐빅 회장형 보기가 그런데 어차피 큐빅 사람 너무 많아서 누군지 다 모를듯 노바 왜 감 나? 엉 끌렸어? 그 소개하러 올때 1등했다길래 어 가야겠다 이름도 멋있고 하 고민해바 오키 으으ㅡ 아니 그럼 약속 다시 잡아야함? 그러지않을까? 짜증나 낼 안되면 화욜에 되도록 추진해야겠다 화욜 ? 아 이번주? 아 그리 치 낼모레 너가 선빵쳤구나 빨리 해버려야지 안구ㅣ차낱 맞긴해 언제 말하나싶었어 쟤네 아 그러네? 공업 화학과는 처음 듣는데 나도 웬만한 건 다아는데 진짜 생소하네 화학과랑 다른가 화공이랑 비슷한가 형진 인디노래 추천좀 좀 빡센거 ㅎㅎ 빡센 인디? Z 어떻게 빡센거 아 박자감 ㅎㅎ 리듬 오케이 기대할게 안돼 기대금지 ㅇㅋ… 검정치마를 아시나요 몰라 오잉 ? 생각보다 안 유명하구나 뭐야 2288만회? 아니 내가 잘 몰라 모른다면 샵검색: #everything 검정치마 일단 이걸 드럼 시작이네 박자감은 잘모르겠는데 그냥 몽환적인 노래야 좋지 혁오 느낌 그치그치 오 이거 어디서 많이 들어본 것 같은데 유명해서 그런가 이게 젤 히트친거라 아마 들어봣을거야 좋다 허회경 알아? 그럼유 노래 2개밖에 모르긴한데 허회경 - 그렇게 살아가는 것 이거야 ? 어 들켰다 아 한개더 아무것도 상관없어 검정치마에서 인디 플리로 넘어가버렸다 ㅎㅎ ㅋㅋ 여성분이시네 맞어 나도 남자인줄 ㅋㅋ 새벽에 들으면 이제 못 벗어나 무서운 장르구만 이제 알고리즘에 지배당할 일만 남았다구 내일 그대로 하겠는데? ㅎㅋ 개뀰 아니 개 에반데 족보랑 똑같은게 말이 됨? 아 억울해 똑같다고? 거의 똑같아 왜 난 다르다고느꼈지 그래?? 나이수 1번 똑같고 2번 숫자만 바꿨고 오 족보 4번이랑 3번이랑 숫자만 다르고 2주차 말하는거지 으 1주차 이거이거 노바 가야겠는데 누나한테 고마워섴ㅋ 노바 사람도 별로없어서 족보는 다 잘 챙겨줄듯 사람 없어?? 딴데보다는 큐빅이나 네넷보단 확실히 적으니까 아 그려? 큐빅 사람 적은 줄 알았네 말하는거보면 꽤많아보이던데'

In [69]:
text = conv.mergeConversation[-1][2]

In [256]:
conv.mergeConversation[24][2]

' 살아있나 살아있네여 근데 독감 걸린듯 아 어제 약간 느낌이 어 딱 이건 몸살이다 하고 머리도 살짝 바로 자지 아프기시작하면서~ 바로 누웠디 지금 아픔?? 그냥 털옷입고 잠 지금은 버틸만해 버텨야지 안그럼 재수강인골~ 교양 결석마렵다 3시간 인데 흠 2시네 맞지 컴구 에바 ㅎㅎ 아 자구까지 같이들어오니까 엉 죽을맛이다 그러겠어 정말 인생 이틀 남은거 자구 컴구 객프? 실화? ㅎㅎ 맞아 내 미랜데 하하 ㅎㅎ 공윤사 도 있다는사실 그건 난 금욜에 그것만 있어서 ㅎㅎ 오 개부럽 목욜이 뒤짐 3개임 상당한걸? ;;그러니까 교양때 자구 딱대 뒤에 자리잡아야지 교양은 수업이 아니더라 ㄹ 하 컴구 연습문제 푸니? ㅠ 연습문제 풀어야할더같아 그른가 ㅎㅎ 가야지가야지 아직도 수업? 쑤업끝낫지 빨리 끝나네 하 어지롭네 하기 싫다 인정드립니다 아 ? 무수한 굴레와 속박에 있는거같다 아니지 자구 개재밋다 내가 다 혼내줘야지~ 미쳤눙 컴구나 답지 하… ㅎㅎ 책에 답지 없으면 근데 공부어케해 ㄹㅇ 진짜 난 모르겠어 이악물고 버티자 오우 앙 ㅎ 낼 동방 감? 내일 안갈거같은디 셤끝나고갈듯 아 낼 선대가 마지막인가? 예압버디 ㅋㅋ 빡공이시네 그러면 좋겠읍니다 나도 내가 빡공하는거 같지 않아 ?? 그럼 어려워요 죽겠어요 자구요? 네 전 컴구요 후욱 컨디션이 지하에 박혀있네요 ㅇㅈ 나도 몇시에 자? 그걸 정해두는건 술 마시다 몇시에 들어갈지 정하는 것과 같아 아하? 너 좀 강하다? 나약합니다 감기나 걸리고 졸린데 잘까 그것도 나쁘지않은선택'

In [253]:
b = conv.mergeConversation[24][2]

In [70]:
kiwi = Kiwi()
result_text = ''
for sentence in kiwi.analyze(b):
    nouns = []
    for token in sentence[0]:
        if token.tag.startswith('NN'):
            nouns.append(token.form)
    if nouns:
        result_text = ' '.join(nouns)
print(result_text)

인터스텔라 스포 인터스텔라 도서관 아빠 엄마 동생 동생 이해 대화 인터스텔라 센 치히로 행방불명 봤 센 치히로 봤지 센 치히로 지브리 지브리 센 치히로 돼지 돼지 비하 신고 신고 돼지 존경 동경 하쿠 돼지고기 멋 쿠 남자 여자 존 남자 남자 거 하쿠 여자 아쉬움 수 조금 대화 뭐였 체육 대회 종목 몸 오 후 삼각근 알 배미 상품 대 체육 대회 개 이득 것 때 거 시간 체육 대회 술 닭발 삼겹살 돈 지리지 원 가성비 지리 입 술 술 술 잔 소오주 볼 것 핫식스 카페인 덩어리 번


In [248]:
result_text = "건영 거 동아리 노바 작년 등 동아리 오늘 상관 노바 애 고민 큐빅 회장 큐빅 사람 듯 노바 소개 때 등 이름 고민 해바 오키 약속 짜증 화욜 추진 화욜 이번 주 낼 모레 빵 안구 차낱 말 쟤네 공업 학과 거 화학과 화공 형 인디 노래 추천 거 인디 거 박자 감 리듬 기대 것 기대 금지 검정치마 오잉 생각 유명 회 샵 검색 검정치마 드럼 시작 박자 몽환 노래 혁오 느낌 것 유명 히트 거 거 허회 경 유 노래 개 허회경 것 개 아무것 검정치마 인디 플리 여성분 남자 인줄 새벽 장르 알고리즘 지배 일 내일 개뀰 개 에바 족보 것 말 나이 수 번 번 숫자 오 족보 번 번 숫자 주 차 말 거 주 차 이거이 노바 누나 노바 사람 족보 듯 사람 데 큐빅 네넷 큐빅 사람 줄 말 거"

In [26]:
s = "근데 난 어디든 상관이 없어서 노바 와 나 외로워 아는 애가없어 진짜로 고민이네 너무 그 큐빅 회장형 보기가 그런데 어차피 큐빅 사람 너무 많아서 누군지 다 모를듯 노바 왜 감 나입니다."

In [27]:
keywords = kw_model.extract_keywords(s, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=30)
keywords

[('외로워', 0.5194),
 ('모를듯', 0.4456),
 ('큐빅', 0.4225),
 ('나입니다', 0.3662),
 ('너무', 0.3496),
 ('사람', 0.3441),
 ('노바', 0.3402),
 ('그런데', 0.3357),
 ('누군지', 0.3352),
 ('아는', 0.3323),
 ('애가없어', 0.3302),
 ('어차피', 0.3141),
 ('어디든', 0.3039),
 ('보기가', 0.2873),
 ('없어서', 0.2848),
 ('진짜로', 0.2846),
 ('많아서', 0.2798),
 ('근데', 0.2771),
 ('고민이네', 0.2703),
 ('회장형', 0.2399),
 ('상관이', 0.2286)]

In [71]:
keywords = kw_model.extract_keywords(result_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=30)
keywords

[('남자', 0.644),
 ('치히로', 0.6357),
 ('아빠', 0.6032),
 ('봤지', 0.5981),
 ('비하', 0.5976),
 ('소오주', 0.5965),
 ('뭐였', 0.5745),
 ('신고', 0.5621),
 ('대화', 0.5482),
 ('가성비', 0.5473),
 ('인터스텔라', 0.5467),
 ('엄마', 0.5424),
 ('아쉬움', 0.5324),
 ('조금', 0.5196),
 ('여자', 0.5006),
 ('돼지', 0.497),
 ('지리지', 0.4955),
 ('하쿠', 0.4865),
 ('종목', 0.4732),
 ('돼지고기', 0.4636),
 ('배미', 0.4588),
 ('이해', 0.4484),
 ('지브리', 0.4446),
 ('지리', 0.4319),
 ('행방불명', 0.429),
 ('삼겹살', 0.4183),
 ('덩어리', 0.412),
 ('존경', 0.3977),
 ('대회', 0.3956),
 ('스포', 0.3702)]

### 요약 4...

In [4]:
import sys
import requests
import json
client_id = " X-NCP-APIGW-API-KEY-ID"
client_secret = " X-NCP-APIGW-API-KEY"
headers = {
    "X-NCP-APIGW-API-KEY-ID": "yukhq4rjut",
    "X-NCP-APIGW-API-KEY": "WobdjgWwkOFR8Qs8hyeGAl9gyh4OG0Je33Ofq34x",
    "Content-Type": "application/json"
}
language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'하루 2000억' 판 커지는 간편송금 시장"
content = '건영 어디갈거야 동아리?? 예아 진짜 모르겠네 노바가 어디였지 작년 1등 한 동아리? 아하?! 으 저거 오늘까지야? ㄴㄴ 근데 난 어디든 상관이 없어서 노바 와 나 외로워 아는 애가없어 진짜로 고민이네 너무 그 큐빅 회장형 보기가 그런데 어차피 큐빅 사람 너무 많아서 누군지 다 모를듯 노바 왜 감 나? 엉 끌렸어? 그 소개하러 올때 1등했다길래 어 가야겠다 이름도 멋있고 하 고민해바 오키 으으ㅡ 아니 그럼 약속 다시 잡아야함? 그러지않을까? 짜증나 낼 안되면 화욜에 되도록 추진해야겠다 화욜 ? 아 이번주? 아 그리 치 낼모레 너가 선빵쳤구나 빨리 해버려야지 안구ㅣ차낱 맞긴해 언제 말하나싶었어 쟤네 아 그러네? 공업 화학과는 처음 듣는데 나도 웬만한 건 다아는데 진짜 생소하네 화학과랑 다른가 화공이랑 비슷한가 형진 인디노래 추천좀 좀 빡센거 ㅎㅎ 빡센 인디? Z 어떻게 빡센거 아 박자감 ㅎㅎ 리듬 오케이 기대할게 안돼 기대금지 ㅇㅋ… 검정치마를 아시나요 몰라 오잉 ? 생각보다 안 유명하구나 뭐야 2288만회? 아니 내가 잘 몰라 모른다면 샵검색: #everything 검정치마 일단 이걸 드럼 시작이네 박자감은 잘모르겠는데 그냥 몽환적인 노래야 좋지 혁오 느낌 그치그치 오 이거 어디서 많이 들어본 것 같은데 유명해서 그런가 이게 젤 히트친거라 아마 들어봣을거야 좋다 허회경 알아? 그럼유 노래 2개밖에 모르긴한데 허회경 - 그렇게 살아가는 것 이거야 ? 어 들켰다 아 한개더 아무것도 상관없어 검정치마에서 인디 플리로 넘어가버렸다 ㅎㅎ ㅋㅋ 여성분이시네 맞어 나도 남자인줄 ㅋㅋ 새벽에 들으면 이제 못 벗어나 무서운 장르구만 이제 알고리즘에 지배당할 일만 남았다구 내일 그대로 하겠는데? ㅎㅋ 개뀰 아니 개 에반데 족보랑 똑같은게 말이 됨? 아 억울해 똑같다고? 거의 똑같아 왜 난 다르다고느꼈지 그래?? 나이수 1번 똑같고 2번 숫자만 바꿨고 오 족보 4번이랑 3번이랑 숫자만 다르고 2주차 말하는거지 으 1주차 이거이거 노바 가야겠는데 누나한테 고마워섴ㅋ 노바 사람도 별로없어서 족보는 다 잘 챙겨줄듯 사람 없어?? 딴데보다는 큐빅이나 네넷보단 확실히 적으니까 아 그려? 큐빅 사람 적은 줄 알았네 말하는거보면 꽤많아보이던데'
content = '나는 인터스텔라 봤어 난 안봤는데 재밌어? 스포해줄까? ㄱㄱ 인터스텔라는 도서관에서 아빠가 나와. 아니 엄마가 아니 뭐였지. 동생 아니 여 동생 이해했어? ㅇㅎ 나만 대화하니? 그러다가 그럴까? 어렵다 인터스텔라 센과 치히로의 행방불명 봤어? 센과 치히로 봤지 센 치히로 나 지브리 좋아하거든 지브리 좋아해? 나는 센과 치히로에 나오는 돼지를 보고 울었어 너무 슬프게 생겨서 그거 돼지 비하아니야? 신고 신고하지마. 나는 돼지를 좋아하며 존경하며 동경해 난 거기 하쿠가 돼지고기 먹고싶네 좀 멋있더라 응 오 하쿠는 남자야 여자야? 존잘 남자 남자구나 몰랐어. 나도 저렇게 태어날걸 하쿠는 여자 닮았더라고 아쉬움이 남아 아하? 그럴 순 없어 우리 조금만 더 대화해야해 어제 뭐였ㅈ 체육대회 갔어? 어제 전 종목 다 나갔지. 온 몸이 다 쑤셔 오 심지어 이겼어? 후면삼각근까지 알 배겼어. 오 이겼지 그래서 진짜? 배민상품권 준대 체육대회 이겼구나?! 개이득 나도 갈게 그땐 언제 올건데 당신 난 시간이 되면 체육대회에서 술도 마시고 닭발도 먹고 삼겹살도 먹었어 돈도 내고? 지리지 만원 냈는데 가성비 지리네 난 한입 밖에 안 먹었어 어머 누가 다 먹었을까. 술은 누가 마셨어 술도 다 마셨나. 난 술 한 잔 마셨어. 소오주? 하이볼 맛있었겠다. 맛있더라 이만하면 될 것 같은데 핫식스랑 섞으면 좀 맛있어짐. 카페인 덩어리네 ㅇㅈ 곧 뒤짐 이걸 한 번 돌려볼게 ㅎㅎ'
data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}
print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

{
    "document": {
        "content": "\ub098\ub294 \uc778\ud130\uc2a4\ud154\ub77c \ubd24\uc5b4 \ub09c \uc548\ubd24\ub294\ub370 \uc7ac\ubc0c\uc5b4? \uc2a4\ud3ec\ud574\uc904\uae4c? \u3131\u3131 \uc778\ud130\uc2a4\ud154\ub77c\ub294 \ub3c4\uc11c\uad00\uc5d0\uc11c \uc544\ube60\uac00 \ub098\uc640. \uc544\ub2c8 \uc5c4\ub9c8\uac00 \uc544\ub2c8 \ubb50\uc600\uc9c0. \ub3d9\uc0dd \uc544\ub2c8 \uc5ec \ub3d9\uc0dd \uc774\ud574\ud588\uc5b4? \u3147\u314e \ub098\ub9cc \ub300\ud654\ud558\ub2c8? \uadf8\ub7ec\ub2e4\uac00 \uadf8\ub7f4\uae4c? \uc5b4\ub835\ub2e4 \uc778\ud130\uc2a4\ud154\ub77c \uc13c\uacfc \uce58\ud788\ub85c\uc758 \ud589\ubc29\ubd88\uba85 \ubd24\uc5b4? \uc13c\uacfc \uce58\ud788\ub85c \ubd24\uc9c0 \uc13c \uce58\ud788\ub85c \ub098 \uc9c0\ube0c\ub9ac \uc88b\uc544\ud558\uac70\ub4e0 \uc9c0\ube0c\ub9ac \uc88b\uc544\ud574? \ub098\ub294 \uc13c\uacfc \uce58\ud788\ub85c\uc5d0 \ub098\uc624\ub294 \ub3fc\uc9c0\ub97c \ubcf4\uace0 \uc6b8\uc5c8\uc5b4 \ub108\ubb34 \uc2ac\ud504\uac8c \uc0dd\uaca8\uc11c \uadf

#### 종합

In [ ]:
conv.mergeConversation[-1][2]

In [60]:
ㅠconv.mergeConversation[-1][2]

' 나는 인터스텔라 봤어 난 안봤는데 재밌어? 스포해줄까? ㄱㄱ 인터스텔라는 도서관에서 아빠가 나와 아니 엄마가 아니 뭐였지 동생 아니 여 동생 이해했어? ㅇㅎ 나만 대화하니? 이게 다 중요하다고 딴걸로 가자 그러다가 그럴까? 어렵다 인터스텔라 센과 치히로의 행방불명 봤ㅇ? 센과 치히로 봤지 센 치히로 나 지브리 좋아하거든 지브리 좋아해? 나는 센과 치히로에 나오는 돼지를 보고 울었어 너무 슬프게 생겨서 그거 돼지 비하아니야? 신고 신고하지마 나는 돼지를 좋아하며 존경하며 동경해 난 거기 하쿠가 돼지고기 먹고싶네 좀 멋있더라 응 오 하쿠는 남자야 여자야? 존잘 남자 남자구나 몰랐어 나도 저렇게 태어날걸 하쿠는 여자 닮았더라고 아쉬움이 남아 아하? 그럴 순 없어 우리 조금만 더 대화해야해 어제 뭐였ㅈ 체육대회 갔어? 어제 전 종목 다 나갔지 온 몸이 다 쑤셔 오 심지어 이겼어? 후면삼각근까지 알 배겼어 오 이겼지 그래서 진짜? 배민상품권 준대 체육대회 이겼구나?! 개이득 나도 갈게 그땐 언제 올건데 당신 난 시간이 되면 체육대회에서 술도 마시고 닭발도 먹고 삼겹살도 먹었어 돈도 내고? 지리지 만원 냈는데 가성비 지리네 난 한입 밖에 안 먹었어 어머 누가 다 먹었을까 술은 누가 마셨어 술도 다 마셨나 난 술 한 잔 마셨어 소오주? 하이볼 맛있었겠다 맛있더라 이만하면 될 것 같은데 핫식스랑 섞으면 좀 맛있어짐 카페인 덩어리네 ㅇㅈ 곧 뒤짐 이걸 한 번 돌려볼게 ㅎㅎ'

In [65]:
b = ''' 나는 인터스텔라 봤어 난 안봤는데 재밌어? 스포해줄까? ㄱㄱ 인터스텔라는 도서관에서 아빠가 나와 아니 엄마가 아니 뭐였지 동생 아니 여 동생 이해했어? ㅇㅎ 나만 대화하니? 그러다가 그럴까? 어렵다 인터스텔라 센과 치히로의 행방불명 봤ㅇ? 센과 치히로 봤지 센 치히로 나 지브리 좋아하거든 지브리 좋아해? 나는 센과 치히로에 나오는 돼지를 보고 울었어 너무 슬프게 생겨서 그거 돼지 비하아니야? 신고 신고하지마 나는 돼지를 좋아하며 존경하며 동경해 난 거기 하쿠가 돼지고기 먹고싶네 좀 멋있더라 응 오 하쿠는 남자야 여자야? 존잘 남자 남자구나 몰랐어 나도 저렇게 태어날걸 하쿠는 여자 닮았더라고 아쉬움이 남아 아하? 그럴 순 없어 우리 조금만 더 대화해야해 어제 뭐였ㅈ 체육대회 갔어? 어제 전 종목 다 나갔지 온 몸이 다 쑤셔 오 심지어 이겼어? 후면삼각근까지 알 배겼어 오 이겼지 그래서 진짜? 배민상품권 준대 체육대회 이겼구나?! 개이득 나도 갈게 그땐 언제 올건데 당신 난 시간이 되면 체육대회에서 술도 마시고 닭발도 먹고 삼겹살도 먹었어 돈도 내고? 지리지 만원 냈는데 가성비 지리네 난 한입 밖에 안 먹었어 어머 누가 다 먹었을까 술은 누가 마셨어 술도 다 마셨나 난 술 한 잔 마셨어 소오주? 하이볼 맛있었겠다 맛있더라 이만하면 될 것 같은데 핫식스랑 섞으면 좀 맛있어짐 카페인 덩어리네 ㅇㅈ 곧 뒤짐 이걸 한 번 돌려볼게 ㅎㅎ'''


In [68]:
import sys
import requests
import json
client_id = " X-NCP-APIGW-API-KEY-ID"
client_secret = " X-NCP-APIGW-API-KEY"
headers = {
    "X-NCP-APIGW-API-KEY-ID": "yukhq4rjut",
    "X-NCP-APIGW-API-KEY": "WobdjgWwkOFR8Qs8hyeGAl9gyh4OG0Je33Ofq34x",
    "Content-Type": "application/json"
}
language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'"
content = conv.mergeConversation[-2][2]
data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}
print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

{
    "document": {
        "content": " \ud615\uc9c4\uc528 \uc2dc\uc2a4\ud15c\uc18c\ud504\ud2b8\uc6e8\uc5b4 \ud560 \ub54c math \ub77c\uc774\ube0c\ub7ec\ub9ac\ub294 \uc0ac\uc6a9\ud574\ub3c4 \ub418\ub294\uac70\uc5d0\uc694? \uc774\uac78 \uc368\ub3c4 \ub418\ub294\uc9c0 \ud655\uc778 \ud558\ub294 \ubc29\ubc95\uc774 \uc788\uc5b4? \uc548\ub420\uac70\uc788\ub098\uc5ec math\ub85c \uc678\ubd80 \ub77c\uc774\ube0c\ub7ec\ub9ac \ubc0f \uc678\ubd80 \ud5e4\ub354\ub294 \uc0ac\uc6a9 \ubd88\uac00\ud569\ub2c8\ub2e4 \ubb50\uc4f0\uc2e4\ub824\uace0 \ub77c\uace0 \ub418\ub358\ub370 sqrt \uc4f0\ub824\uace0 \ud569\ub2c8\ub2e4 math\uac00 \ub0b4\uc7a5\uc778\uac00 \uadf8\uac78 \ubd10\uc57c\ud558\ub098? \uadf8\ub798\uc57c\ud560\ub4ef\ud55c\ub370 \uadfc\ub370 math\ub294 \uc678\ubd80\uac19\uc9c4 \uc54a\uc740\ub370 \ud45c\uc900 \ub77c\uc774\ube0c\ub7ec\ub9ac\ub798 \uadf8\ub7fc \ub41c\uac74\uac00? \ub418\uaca0\uc9c0 \ubb50 \uc678\ubd80\ub294 \ubb54\uac00 \ub354 \uc774\uc0c1\ud55c \ub77c\uc774\ube0c\ub7ec\ub9ac\ucc98\ub7

In [3]:
g = """최근에 면접 들어갔다가 나오면서 문득 아 이제 면접관으로 들어가는 것도 좀 익숙해진 것 같다 라는 느낌을 받았다. 그 김에 되짚어 보니, 팀에서 분석가를 뽑는 데 처음 참여하기 시작했던 게 딱 1년 전쯤이었다. 처음에는 서류/과제 검토하고 1차 면접(흔히 말하는 직군 면접 or 기술 면접) 위주로 들어갔었고, 지금은 팀 리더로 전체적인 프로세스에 관여를 하고 있다.

1년 전의 나는 연차가 별로 안 많은 건 둘째치고, 인턴 했던 회사에 계속 다니면서 이직 경험도 없는 케이스기 때문에 면접관은커녕 지원자로서 면접 경험 자체도 매우 부족했다. 그래서 처음에는 제가..면접을 들어가라고요..? 제가 그럴 레벨인가요..? 하는 마음이었다.

하지만 모든 것이 그렇듯이 역시 하다 보면 익숙해진다! 그리고 어쨌든 함께 일할 좋은 분을 모시는 것은 매우 중요한 일이므로 어떻게 하지..(패닉) 보다는 어떻게 (잘) 하지? 라는 생각을 하는 게 맞았다. 처음에는 다른 면접관 분들이 어떻게 진행하고 질문하시는지 미어캣처럼 열심히 보고 그랬다.

아무튼 1년이 된 김에 그간 채용에 참여하면서 느낀 점들을 한번 간단히 정리해 보려고 한다. 요즘 업계 전반적으로 채용이 좀 침체된 것 같은데 그 와중에도 데이터 분석가를 뽑아야 하는 팀이나 지원자 입장에서 조금 도움이 되기를 바라며…? 물론 아래 적을 모든 내용은 나 개인의 경험에 기반한 것이므로 감안하고 보면 된다. 회고성 글에 가까우므로 평소 글과 달리 편한 말투로 적는다.

들어가기 전에 약간의 배경설명: 나는 게임 데이터 기반의 솔루션(데이터 프로덕트)를 만드는 데이터 분석가로 이루어진 팀에서 일하고 있다. 상시 채용을 하고 있고, 일반적으로 서류 - 과제 - 2차례 면접의 프로세스로 진행한다. 우리 팀 분석가들 업무는 주로 지표와 모델 개발이고 어디 가서 하는 일을 설명하면 데이터 분석가보다는 데이터 사이언티스트 아닌가요? 라고 하지만, 개인적으로는 데이터 사이언티스트라는 말이 좀 애매하다고 생각하며 모델링보다는 문제 정의와 분석을 잘 할 줄 아는 분석가가 되고자 하고 그런 분석가와 같이 일하는 것을 선호한다.

참여하는 팀 구성원들
가능하다면 (팀이 충분히 크다면…) 여러 명의 분석가로 이루어진 미니 채용 팀을 꾸리는 게 좋다! 왜냐면
각 팀원 입장에서는 채용도 하나의 업무이다.
서류나 과제를 평가하는 데도 시간이 들어가며, 면접의 경우 1시간 본다 치면 앞서서 준비하는 시간+끝나고 논의하는 시간까지 2시간 이상이 소요된다. 수시채용의 경우 한참 서류가 많이 들어오지 않다가 어떤 때는 와르르 들어오기도 하는데 만약 굉장히 소수의 인원이 이걸 전담해서 본다고 한다면 굉장히 빡셀 것이다. 면접도 면접에 들어갈 수 있는 분석가 풀을 4~5명으로 두고 그때그때 가능한 사람이 나눠서 들어간다면 부담을 줄일 수 있다.
여러 명의 의견은 대체로 도움이 된다.
아무리 평가 기준의 컨센서스를 맞춰 놨다고 한들 정작 매 단계에서 각자의 의견은 조금씩 다르다. 같은 자질을 두고도 어떤 사람은 긍정적으로 보고 다른 사람은 단점이라고 생각할 수 있다.
이걸 하나의 의견으로 수렴하는 게 힘들지 않나 라고 생각할 수도 있지만, 내가 생각하는 이 방식의 가장 좋은 점은 내가 편협하게 보아 놓칠 수 있던 지원자를 놓치지 않게 해준다는 것이다. 실제로 과제 기준 나는 탈락 의견이었던 지원자 분을 면접관 한 분이 좋게 보아 면접을 보았는데 면접에서는 나도 긍정적인 평가를 했던 적이 있다.
어쨌든 같이 채용에 참여하는 팀원들끼리는 서로의 판단을 신뢰하고 있기 때문에, 내가 별로라고 생각했던 지원자도 다른 사람이 매우 긍정적으로 평가한다면 한 번 더 볼 의지가 있다.
어떤 사람을 원하는지 확실히 하기 (근데 빨리 하기)
좀 구린 비유를 하자면 소개팅을 주선한다고 치자. 어떤 사람이 좋냐고 물었을 때 그냥 뭐 다 괜찮아 라고 두루뭉술하게 대답하는 친구보다는 나는 키 xx 이상 나랑 비슷한 업계 다니는 사람 만나고 싶어 라고 말하는 친구가 더 시켜주기 쉽다. 그리고 실제로 만나서도 빠르게 아 내가 원한 건 이런 사람이 아냐 하고 그만 만날수록 양측이 편하다. 채용도 딱 그런 것 같다.
어떤 사람을 원한다의 문제는 이 분이 실제로 팀에 들어 왔을 때 어떤 일을 하기를 원하는지와도 직결된다. 빠르게 실무에 적응해서 일손을 줄여줄 사람을 원할 수도 있고, 기존 인원들이 하지 못했던 새로운 방법으로 기존에 산재해 있던 문제를 해결해 줄 사람을 원할 수도 있다. 아니면 가장 간단하게 이미 있던 구성원이 나가서 그 사람이 하던 일을 대체해 줄 사람을 원할 수도 있다.
물론 무슨 일을 맡겨도 다 잘하는 올라운더를 뽑고 싶어! 라고 할 수도 있지만 세상에 그런 지원자가 얼마나 있겠어요. 모든 사람은 강점과 약점이 있고 강점이 우리 팀 빈 자리에 맞아야 함.
어떤 사람을 원하는지는 당연히 미리 공고 JD에도 일부 표현이 되어 있어야 하고 팀 내부에도 협의가 있어야 한다. 채용 전형이 뒤로 갈수록 점점 구체적으로 지원자가 그런 사람인지 검증하고, 사실 마지막에는 지원자에게도 우리가 원하는 것이 어느 정도 전달이 되어야 한다(채용 전형은 지원자도 우리를 평가하는 과정이기 때문에).
아무리 훌륭한 지원자라 한들 지금 우리가 원하는 목적과 맞지 않으면 서로에게 시간낭비인데, 이걸 전형이 많이 진행되기 전에 빨리 깨달을수록 좋다.
개인적으로 내가 어려웠던 것은 1) 우리 팀에서 필요한 사람이 딱 한 가지 유형이 아니고 2) 전형이 더 진행되기 전에 이 지원자가 어떤 유형인지를 명확히 파악하기가 어렵기 때문이다. 면접 보기 전에 면접관들이 모여서 제일 먼저 하는 말이 이 분 (만약 입사한다면 팀 내에서) 어떤 역할로 생각하세요? 인데 그것에 답하기가 항상 쉽지는 않다.
서류와 과제
최소한 서류나 과제에 한해서는 평가 기준을 세우고 결과를 정량화하는 게 도움이 된다. 우선 정량화할 기준을 세우기 위해서 팀원들이 모여서 논의를 하는 과정에서 앞서 말한 우리가 어떤 사람을 원하는지가 구체적으로 정립이 되어서 좋다. 정리가 되고 나서부터는 해당 기준에 따라서 평가를 하고 합불을 결정하는데, 체크리스트를 체크하는 수준이기 때문에 그냥 이 사람 이러저러해서 합격시키자 하고 의견을 내는 것에 비해 사실상 인지적인 자원도 훨씬 덜 들어간다.
과제를 보는 이유는 과제를 통해 생각보다 정말 많은 것들을 알 수 있기 때문이다. 기본적으로 분석/코딩 역량이 되는지 스킬 수준의 능력부터 논리적인 전개나 문서 전달력까지 볼 수 있으므로 면접 때 기초적인 기술 역량을 검증하는 데 시간을 덜 쓸 수 있다.
다만 어떤 과제(문제)가 좋은 과제이냐에 대해서는 아직까지도 고민이 많다. 다시 어떤 사람을 원하는지로 돌아가서, 팀별로 과제도 목적에 맞게 출제가 되어야 한다. 출제 시에 trade-off가 발생할 수 있는 부분은 과제 문제나 지시문이 구체적이고 명확할수록,
(+) 지원자가 과제를 포기할 확률이 낮고 (잘 설계되었다면) 우리가 보고자 하는 걸 볼 수 있다.
(-) 지원자 스스로 문제 정의를 하고 분석을 설계하는 능력을 보기는 어려울 수 있으며 모든 과제 결과물이 찍어낸 듯 천편일률적인 슬픈 상황을 맞닥뜨릴 수도 있다.
면접
모든 면접 전에는 면접관들이 모여서 이 지원자 분에게 어떤 내용이 궁금한지(어떤 질문들을 할지) 정리하고 들어간다. 하지만 그 전에도 이미 주로 물어보는 질문 리스트와 흐름이 정해져 있으면 좋다. 면접이야말로 상황과 지원자에 따라 변수가 많은 영역이고, 그냥 꼬리물기 질문으로 30분이 지나갈 때도 있지만.
다음은 그냥 면접 보면서 내가 느낀 점과 고민들이다.
기본적인 것은 어느 정도까지 물어봐야 하나?
다른 거 물어볼 거 많은데 너무 많이 시간 쓰면 낭비 아닌가 싶지만 놀랍게도 가끔 이력서에 XX(진짜 기본 ML 지식이 있으면 아는 모델)를 사용해서 프로젝트를 했다고 썼는데 XX가 뭔지 설명해 달라고 하면 설명 못하시는 분이 있다.
그리고 당연히 알 것 같은 개념이나 모델이라도, 질문을 함으로써 지원자 분의 설명하시는 능력을 볼 수 있다. 나는 이 개념을 제가 통계알못이라고 치고 저한테 설명해주세요 라는 질문도 종종 한다. 왜냐면 실제 우리의 고객은 대부분 통계알못이므로..
긴장 풀어드리기
내가 잘 못하는 영역으로 거의 한번도 적극적으로 해본 적은 없다. 약간이라도 웃음 포인트가 생길 때 최대한 웃어드리기가 끝…
가끔 신입 면접인 경우 너무 긴장하시는 경우가 있는데, 같이 들어간 면접관 분들이 이걸 잘하시는 걸 보면 배워야지 한다. 지원자 분이 긴장해서 해야 할 말도 못하시고 충분히 능력을 표현하지 못하면 우리도 손해이다.
면접의 목적이 뭘까
서류와 과제를 통해 이미 내 머릿속에 어느 정도 파악된 지원자의 특성이 있고, 그 특성이 좋으니까 면접을 보기로 한 것이다. 이제 면접은 그걸 뒷받침하는 근거를 얻을 수 있는 자리이다. 면접관이 그 근거를 잘 파낼 수 있는 질문을 던지도록 노력하고 지원자가 앞선 전형과 일관성 있는 모습을 보여준다면 면접이 어느 정도 잘 흘러간다고 생각한다. 일관성 있는 모습이라면 예를 들어 이런 건데, 지원자가 서류에 나는 꾸준하게 새로운 걸 학습하는 사람이라고 적었다면 최근에 공부한 거 뭐 있어요? 소개해주세요 라는 질문에 뭐라도 답이 나와야 한다.
커뮤니케이션
오직 면접에서만 검증할 수 있는 능력이다. 우선 질문이 오고 갈 때 막히는 구석 없이 매끄럽게 흘러가는지가 중요한데(질문의 요지를 잘 파악하고 적절한 대답을 하는지), 당연히 모든 말을 아무 버벅거림 없이 서로 알아듣기란 어려울 것이다. 질문을 잘 못 알아들었다면 다시 한 번 말해달라고 하거나 내가 이렇게 이해한 게 맞는지 되묻는 것 아주 좋다. 업무를 할 때도 그렇게 해야 하기 때문이다(못 알아들었는데 대충 알아들은 척 답변하는 것 = 그렇게 일을 진행해버리면 보통 망함).
그리고 이력서 상에 있는 내용에 대해 어떻게 진행했고 어떤 역할을 했는지 설명해달라고 했을 때 그 내용 자체만큼이나 어떤 방식으로 설명하는지를 보는 것도 중요하다. 너무너무 당연한 소리지만 우리 팀 아니라 세상의 어떤 회사에서도 분석가라고 분석만 하는 경우는 없기 때문이다. 분석한 내용을 설명해야 하고, 설명을 잘 해서 남들을 설득해야 하는데, 보통 그 남들에는 내가 개떡같이 말해도 찰떡같이 알아듣는 같은 분석가 동료만 있는 것이 아니고 나에겐 간단한 용어조차 익숙하지 않은 다른 직군의 사람이나 봐야 할 프로젝트가 오백 개라 내 작은 프로젝트에 쏟을 시간은 별로 없는 상위 리더 등이 포함되어 있으므로 사실 핵심적인 능력이라 할 수 있겠다.
‘바람직한’ 답변
흔히 말하는 social desirabilty가 엄청 높으신 분들이 있다. 물론 지원자라면 당연히 면접관이 원하는 대답을 하고 싶어 한다. 나라도 그럴 것 같다. 하지만 정말 분석가로서의 성향이나 선호, 커리어의 지향을 묻는, 답이 정해져 있지 않은 질문에서 모조리 그런 방향으로 대답하는 건 좋지 않다. 그걸로 그 분이 별로라고 생각한다는 게 아니라, 질문을 아무리 해도 알아낼 수 있는 게 별로 없어서 좋지 않다는 것이다.
물론 쪼렙 면접관인 나의 스킬 탓일 수도 있다. 이 분에 대해서는 이걸 꼭 검증하고 싶다는 내용이 있을 때 그걸 안 들키고(?) 알아내기가 참 어렵다. 예를 들어 우리는 A를 하는 사람을 원하는데 상대방이 B 쪽에 흥미가 있고 강점이 있는 사람인 것 같아서 조금 걸린다고 치면, 최악은 이런 질문인 것 같다. B 쪽을 주로 해오셨고 재밌다고 하셨는데 뭐 다른 것들도 있잖아요, A 도 있고… 어떤 게 더 중요하다고 생각하세요? 그러면 조금이라도 눈치가 있는 사람은 바로 A 좋죠 라는 식으로 방향을 틀어버리고 어디까지가 그 분의 진짜 생각인지를 알 수가 없다. 지원자가 (B는 알겠고) A도 잘 할 줄 아는 사람일지 궁금해! 라는 내 목적을 전혀 달성할 수 없는 것이다.
마지막 질문 타임
당연히 모든 면접 끝에는 지원자 분에게 면접관들에게 궁금한 걸 물어봐 달라고 한다. 이 시간은 지원자 입장에서 회사와 팀에 대해 궁금한 걸 알아갈 수 있을 뿐더러 본인의 관심이나 능력까지도 어필할 수 있는 시간이라고 생각한다.
일단 질문을 하나 정도 의례적으로 하시고 더 궁금한 게 없다고 하시면 확실히 좀 아쉽다. 아니 데이트를 해도 나만 상대방한테 계속 물어보면 얜 나한테 관심이 없구나 이건 글렀구나 하지 않겠나. 이럴 땐 사실 진짜 궁금한 거 더 없으세요? 해도 될 법하지만, 이 단계에 와서 면접 결과까지 뒤집히기는 어렵기 때문에 여러 이유로 굳이 그렇게는 안 하는 듯.
질문의 내용이라고 치면 가장 기본적인 게 입사해서 할 일이 어떤 일이고 어떤 환경(팀 구성, 데이터나 인프라, 업무 체계나 방식 등)일지가 있는데, 이런 질문을 신입일수록 많이 안 하신다. 당연히 궁금할 내용인데 왜 안 하시는지 의문이다.
그 외에도 우리 도메인에 대한 나름의 생각이나 고민을 하시고 그것에 대해 우리 의견을 물어보는 분들이 종종 계신데 그 자체로 1) 그만큼 진짜 이 일에 관심이 있다 2) 이 정도까지 고민할 수 있는 사람이다를 보여주는 것이라서 좋게 보인다.
이 시간에 면접 자체나 지원자로서 본인에 대한 피드백을 달라고 하는 건 뭐 할 만한 질문이라고 생각하지만, 개인적으로 앞으로 분석가가 되기 위해 어떤 걸 공부하고 준비하면 좋겠는지를 물어보는 건 상황에 적절한 질문인지는 잘 모르겠다. 물론 물어보시면 답은 해 드린다.
"""

In [4]:
import sys
import requests
import json
client_id = " X-NCP-APIGW-API-KEY-ID"
client_secret = " X-NCP-APIGW-API-KEY"
headers = {
    "X-NCP-APIGW-API-KEY-ID": "yukhq4rjut",
    "X-NCP-APIGW-API-KEY": "WobdjgWwkOFR8Qs8hyeGAl9gyh4OG0Je33Ofq34x",
    "Content-Type": "application/json"
}
language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'"
content = g
data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}
print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

{
    "document": {
        "content": "\ucd5c\uadfc\uc5d0 \uba74\uc811 \ub4e4\uc5b4\uac14\ub2e4\uac00 \ub098\uc624\uba74\uc11c \ubb38\ub4dd \uc544 \uc774\uc81c \uba74\uc811\uad00\uc73c\ub85c \ub4e4\uc5b4\uac00\ub294 \uac83\ub3c4 \uc880 \uc775\uc219\ud574\uc9c4 \uac83 \uac19\ub2e4 \ub77c\ub294 \ub290\ub08c\uc744 \ubc1b\uc558\ub2e4. \uadf8 \uae40\uc5d0 \ub418\uc9da\uc5b4 \ubcf4\ub2c8, \ud300\uc5d0\uc11c \ubd84\uc11d\uac00\ub97c \ubf51\ub294 \ub370 \ucc98\uc74c \ucc38\uc5ec\ud558\uae30 \uc2dc\uc791\ud588\ub358 \uac8c \ub531 1\ub144 \uc804\ucbe4\uc774\uc5c8\ub2e4. \ucc98\uc74c\uc5d0\ub294 \uc11c\ub958/\uacfc\uc81c \uac80\ud1a0\ud558\uace0 1\ucc28 \uba74\uc811(\ud754\ud788 \ub9d0\ud558\ub294 \uc9c1\uad70 \uba74\uc811 or \uae30\uc220 \uba74\uc811) \uc704\uc8fc\ub85c \ub4e4\uc5b4\uac14\uc5c8\uace0, \uc9c0\uae08\uc740 \ud300 \ub9ac\ub354\ub85c \uc804\uccb4\uc801\uc778 \ud504\ub85c\uc138\uc2a4\uc5d0 \uad00\uc5ec\ub97c \ud558\uace0 \uc788\ub2e4.\n\n1\ub144 \uc804\uc758 \ub098\ub294 \uc5f0\ucc28